In [ ]:
# making all essential imports
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import torch
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Input
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
#load datasets from ACLED (note: you will need to create an ACLED account and generate an access key)
data1 = pd.read_csv(r'/content/ACLED Nigeria 2019-2020.csv')
data2 = pd.read_csv(r'/content/ACLED Nigeria 2019-2020.csv')
df = pd.concat([data1, data2], axis=0)
df.columns

Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

In [ ]:
#datafram with notes and unique IDs
df = df[['event_id_no_cnty','notes']]
df.event_id_no_cnty = df.event_id_no_cnty.astype(str)
df = df.reset_index(drop = True)
df.notes[2379]

'On 2 December 2019, a joint team of Police and Military raided the house of an IPOB lawyer in Orifite and killed two IPOB members. Youth of the town mobilized and clashed with the security forces. Two police officers were killed in the battle.'

In [ ]:
# create a named entity recognition function
def notes_to_entities(notes):
    words = nltk.word_tokenize(notes)
    tag = nltk.pos_tag(words)
    chunks = nltk.ne_chunk(tag, binary=False)
    entities =[]
    labels =[]
    for chunk in chunks:
        if hasattr(chunk,'label'):
            entities.append(' '.join(c[0] for c in chunk))
            labels.append(chunk.label())
        
    entities = ','.join(entities)
    return entities

In [ ]:
# convert notes column to a list of entities
ent_list = []
for note in df.notes:
    temp = notes_to_entities(note)
    ent_list.append(temp)
df['entities'] = ent_list

In [ ]:
df['entities']

0                                                 Katsina
1               Boko Haram,JAS,Wurogayadi,Michika,Adamaya
2                                              Ameke Ngwo
3                     Boko Haram,JAS,Kopa,Michika,Adamaya
4       Nigerian,Boko Haram,JAS,ISWAP,Gujeri,Konduga,B...
                              ...                        
4427    Nigerian,ISWAP,Baga,Goniri,Damasak,Cross Kauwa...
4428    Nigerian,ISWAP,Baga,Goniri,Damasak,Cross Kauwa...
4429    Nigerian,ISWAP,Baga,Goniri,Damasak,Cross Kauwa...
4430    ISWAP,Nigerian,Buni Gari,Gujba,Yobe,Kimba,Biu,...
4431    ISWAP,Nigerian,Buni Gari,Gujba,Yobe,Kimba,Biu,...
Name: entities, Length: 4432, dtype: object

In [ ]:
# pre process entities column
df['entities'] = df.entities.apply(lambda x: x[0:-1].split(','))

In [ ]:
# convert entities column to a list of lists
ent_list = df['entities']
entities = []
for i in range(len(ent_list)):
  entities.append(ent_list[i]) 
entities

[['Katsin'],
 ['Boko Haram', 'JAS', 'Wurogayadi', 'Michika', 'Adamay'],
 ['Ameke Ngw'],
 ['Boko Haram', 'JAS', 'Kopa', 'Michika', 'Adamay'],
 ['Nigerian', 'Boko Haram', 'JAS', 'ISWAP', 'Gujeri', 'Konduga', 'Born'],
 ['Ilorin', 'Senat'],
 ['Nigerian',
  'Boko Haram',
  'JAS',
  'Abulam',
  'Damboa',
  'Borno',
  'Equipment',
  'B'],
 ['ISWAP', 'Mandaragirau', 'Biu', 'Borno', 'Christia'],
 ['Ajow'],
 [''],
 ['Umusadeg'],
 ['APC Edo', 'Jatt'],
 ['INEC', 'Yol'],
 ['Fulani', 'Maisan'],
 ['Fulani', 'Daraw'],
 ['Fulani', 'Madagu', 'Gefen Kubewa', 'Farar Kasa', 'Dutsin M'],
 [''],
 ['Imiakab'],
 ['Imiakab'],
 ['Fulani', 'Tamawa', 'Jallawa', 'Kurfi', 'Kurfi', 'Cattl'],
 ['IM', 'Sokot'],
 ['Fulani', 'Jagbanda', 'Zurmi', 'Zurmi', 'Nigerian Arme'],
 ['Nigerian', 'Armed Forces', 'Tafana', 'Bama', 'Borno', 'Boko Haram', 'JA'],
 ['Fulani', 'Tiv', 'Gonori', 'Gassol', 'Gasso'],
 ['Calaba'],
 ['Onyoho Nike', 'Enug'],
 ['Nigerian',
  'Boko Haram',
  'JAS',
  'Sambisa',
  'Gwoza',
  'Borno',
  'Boko Haram

In [ ]:
# using gensim import word2vec and vectorize the entities list
from gensim.models import Word2Vec
word2vec = Word2Vec(entities, min_count=1)
word2vec

# saving all the vocabulary used in word2vec
vocabulary = list(word2vec.wv.vocab)

In [ ]:
# create a text sequence of text 3 to predict 3rd entity based on the first 2
train_len = 3
text_sequences = []
for i in range(train_len,len(vocabulary)):
  seq = vocabulary[i-train_len:i]
  text_sequences.append(seq)
text_sequences

[['Katsin', 'Boko Haram', 'JAS'],
 ['Boko Haram', 'JAS', 'Wurogayadi'],
 ['JAS', 'Wurogayadi', 'Michika'],
 ['Wurogayadi', 'Michika', 'Adamay'],
 ['Michika', 'Adamay', 'Ameke Ngw'],
 ['Adamay', 'Ameke Ngw', 'Kopa'],
 ['Ameke Ngw', 'Kopa', 'Nigerian'],
 ['Kopa', 'Nigerian', 'ISWAP'],
 ['Nigerian', 'ISWAP', 'Gujeri'],
 ['ISWAP', 'Gujeri', 'Konduga'],
 ['Gujeri', 'Konduga', 'Born'],
 ['Konduga', 'Born', 'Ilorin'],
 ['Born', 'Ilorin', 'Senat'],
 ['Ilorin', 'Senat', 'Abulam'],
 ['Senat', 'Abulam', 'Damboa'],
 ['Abulam', 'Damboa', 'Borno'],
 ['Damboa', 'Borno', 'Equipment'],
 ['Borno', 'Equipment', 'B'],
 ['Equipment', 'B', 'Mandaragirau'],
 ['B', 'Mandaragirau', 'Biu'],
 ['Mandaragirau', 'Biu', 'Christia'],
 ['Biu', 'Christia', 'Ajow'],
 ['Christia', 'Ajow', ''],
 ['Ajow', '', 'Umusadeg'],
 ['', 'Umusadeg', 'APC Edo'],
 ['Umusadeg', 'APC Edo', 'Jatt'],
 ['APC Edo', 'Jatt', 'INEC'],
 ['Jatt', 'INEC', 'Yol'],
 ['INEC', 'Yol', 'Fulani'],
 ['Yol', 'Fulani', 'Maisan'],
 ['Fulani', 'Maisan', 'Dar

In [ ]:
# seperating training sequences and labels. The first 2 entities are being used to predict the third. Think of this as X_train and Y_train

train_sequence = [[word2vec[ingredient] for ingredient in sequence[:2]] for sequence in text_sequences]
train_sequence

label_sequence = [[word2vec[ingredient] for ingredient in sequence[2:]] for sequence in text_sequences]
label_sequence

[[array([-1.6321084e-02,  2.2183897e-02, -2.9497202e-02,  5.1395409e-03,
         -2.4151878e-02,  3.7997838e-02, -7.6626451e-03, -4.8634812e-02,
          2.6295429e-02, -2.4277043e-02, -2.3608889e-02,  8.6988676e-03,
         -1.5645195e-02,  8.9068599e-03,  4.0815793e-02,  2.3875391e-02,
         -9.1312290e-04, -3.0002192e-02,  3.6592036e-02,  3.0412687e-02,
         -1.1015499e-02, -9.2842765e-03, -2.1058813e-02, -3.8650878e-02,
          2.1184389e-02,  2.9482499e-02, -1.2970432e-02,  1.5195178e-02,
          1.1306839e-02,  6.5432461e-03, -3.2822881e-02, -3.2379270e-02,
         -1.5518133e-02, -1.9019289e-02, -1.5978603e-02,  4.5690425e-03,
         -1.0009521e-02,  2.1353912e-02,  6.2464527e-03,  2.4564078e-02,
         -3.2673050e-03,  1.8965818e-02,  4.2125575e-02,  2.7427092e-02,
          2.2302377e-03,  4.2738055e-04, -4.8865832e-02,  3.1882681e-02,
         -3.2100950e-03,  1.7801872e-02,  7.3413321e-05, -3.2213230e-02,
          1.5243557e-02, -2.9286183e-02,  1.7182659

In [ ]:
np.array(train_sequence).shape

(2273, 2, 100)

In [ ]:
np.array(label_sequence).shape

(2273, 1, 100)

In [ ]:
# converting lists to numpy array for feeding to below neural net
train_inputs = np.array(train_sequence)
train_targets = np.array(label_sequence)
train_targets = train_targets.reshape(train_targets.shape[0],train_targets.shape[2])
train_targets.shape, train_inputs.shape

((2273, 100), (2273, 2, 100))

In [ ]:
# setting up the neural net with 2 LSTM and 2 Dense Layers
model = Sequential()
model.add(Input(shape=(2,100)))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(100, activation='relu'))

opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    name="Adam"
)
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 2, 100)            80400     
                                                                 
 lstm_1 (LSTM)               (None, 50)                30200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 100)               5100      
                                                                 
Total params: 118,250
Trainable params: 118,250
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model training on inputs 
history = model.fit(train_inputs,train_targets,epochs=100, validation_split=0.15)

Epoch 1/100
61/61 [==============================] - 6s 24ms/step - loss: 2.0944e-05 - accuracy: 0.0699 - val_loss: 7.1272e-05 - val_accuracy: 0.0674
Epoch 2/100
61/61 [==============================] - 0s 8ms/step - loss: 2.0922e-05 - accuracy: 0.0730 - val_loss: 7.1425e-05 - val_accuracy: 0.0880
Epoch 3/100
61/61 [==============================] - 1s 9ms/step - loss: 2.0894e-05 - accuracy: 0.0740 - val_loss: 7.0277e-05 - val_accuracy: 0.0674
Epoch 4/100
61/61 [==============================] - 0s 8ms/step - loss: 2.0852e-05 - accuracy: 0.0688 - val_loss: 7.0659e-05 - val_accuracy: 0.0674
Epoch 5/100
61/61 [==============================] - 0s 8ms/step - loss: 2.0848e-05 - accuracy: 0.0657 - val_loss: 6.9629e-05 - val_accuracy: 0.0880
Epoch 6/100
61/61 [==============================] - 0s 8ms/step - loss: 2.0877e-05 - accuracy: 0.0657 - val_loss: 6.9040e-05 - val_accuracy: 0.0645
Epoch 7/100
61/61 [==============================] - 0s 8ms/step - loss: 2.0724e-05 - accuracy: 0.0719 - 

In [ ]:
def get_linked_entities(input_news):
    text = notes_to_entities(input_news)
    input_text = text[:].split(',')
    input_text = input_text[0:2]

    # converts text to vector
    input_vector = [word2vec[idx] if idx in word2vec else np.zeros((100,)) for idx in input_text]
    if len(input_vector)!=2:
      while len(input_vector)!=2:
        input_vector.append(np.zeros((100,)))

    # converts input vector to numpy array
    input_vector = np.array([input_vector])

    # # getting model predictions for top 10 suggestions by finding the 10 most similar ingredients to our output
    output_vector = model.predict(input_vector)
    pred = word2vec.most_similar(positive=[output_vector.reshape(100,)], topn=5)
    print("Top 5 linked entities:")
    return pred

In [ ]:
get_linked_entities('On 31 December 2019, Nigerian Air Force conducted air strikes against a camp of Boko Haram (JAS) in Abulam (Damboa, Borno). Equipment was destroyed and an unreported number of BH fighters killed. Unknown number of fatalities coded as 10.')

Top 5 linked entities:


[('Wonaka', 0.6484962701797485),
 ('Zamfara', 0.6471939086914062),
 ('Iperindo', 0.6402360200881958),
 ('Kwaki', 0.6361875534057617),
 ('Katsina', 0.633989691734314)]